In [1]:
import tensorflow as tf
import Batch as bh
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

pathdataSet = 'D:/homework/pre/dataSetSamplerfilterFeature.csv'
pathSaveModel = 'D:/homework/pre/net/mynet'
pathTrainLogs = 'D:/homework/pre/logs/train'
pathTestLogs = 'D:/homework/pre/logs/test'

#划分训练集和测试集比例
testSize = 0.4

dataSet = pd.read_csv(pathdataSet)
dataSet = shuffle(dataSet)
dataTrain , dataTest = train_test_split(dataSet, test_size =testSize)

m,n = dataTrain.shape
dataTrain = bh.Dataset(dataTrain)
dataTest = bh.Dataset(dataTest)


#模型参数设置
lr=0.01
#这里不要超过test的最大样本数数
batch_size=5000               #每一类大概抽取50个样本
training_iters = 3000         #最大迭代次数
n_inputs= n-1                #每个inputs大概有28个特征
n_steps = 1                   #这里没有涉及到时间序列
# n_inputs = 1
# n_steps = n-1
n_hidden_units=20           #隐藏层个数
n_classes=12                #类别数


with tf.name_scope('Input'):
    x=tf.placeholder(tf.float32,[None,n_steps,n_inputs],name="x")
    y=tf.placeholder(tf.float32,[None],name="y")
#同时定义输入和输出的weights和biases
weights={
        'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
        'out':tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
        }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
        'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))
        }

y_one_hot = tf.one_hot(tf.cast(y,tf.int32),n_classes,on_value=1,off_value=None,axis=1)

with tf.name_scope('LstmModel'):
    def RNN(X,weights,biases):
        X=tf.reshape(X,[-1,n_inputs])
        X_in=tf.matmul(X,weights['in'])+biases['in']
        X_in=tf.reshape(X_in,[-1,n_steps,n_hidden_units])
        #使用basicLSTMcell，n_hidden表示神经元的个数，forget_bias就是LSTM们的忘记系数，如果等于1，
        #就是不会忘记任何信息。如果等于0，就都忘记。state_is_tuple使用True，就是表示返回的状态用一个元组表示，
        #对lstm来说，返回的状态是一个包含两个元素的元组：(c_state,h_state) 
        lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)
        #初始化全零state，batch_size就是输入样本批次的数目，dtype就是数据类型。
        _init_state=lstm_cell.zero_state(batch_size,dtype=tf.float32)
        #使用tf.nn.dynamic_rnn(cell, inputs)要确定inputs的格式，time_major参数会针对不同inputs格式有不同的值.
        #1.如果 inputs 为 (batches, steps, inputs) ==> time_major=False;
        #2.如果 inputs 为 (steps, batches, inputs) ==> time_major=True;
        #我们这里是用的第一种（看placeholder，steps在第二个，所以要设为False
        outputs,states=tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=_init_state,time_major=False)
        results=tf.matmul(states[1],weights['out']+biases['out'])
        return results


pred=RNN(x,weights,biases)

with tf.name_scope('loss'):
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_one_hot,logits=pred))
    tf.summary.scalar('loss',cost)

with tf.name_scope('train_step'):
    train_op=tf.train.AdamOptimizer(lr).minimize(cost)
    
with tf.name_scope('Accuracy'):
    correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y_one_hot,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32),name="Accuracy")
    tf.summary.scalar('Accuracy',accuracy)

init=tf.global_variables_initializer()
merged=tf.summary.merge_all()
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    writer_train = tf.summary.FileWriter(pathTrainLogs,sess.graph)
    writer_test = tf.summary.FileWriter(pathTestLogs,sess.graph)

   
    
    step=0
    while step<training_iters:
        dataSet = dataTrain.next_batch(batch_size)
        batch_train_xs = np.array(dataSet.drop(['label'],axis = 1))
        batch_train_xs = batch_train_xs.reshape([-1,n_steps,n_inputs])
        batch_train_ys = np.array(dataSet['label'])
        sess.run([train_op],feed_dict = {x:batch_train_xs,y:batch_train_ys})
        
        if step%10==0:
            print('step'+str(step)+':')
            print('Train accuracy: ',sess.run(accuracy,feed_dict={x:batch_train_xs,y:batch_train_ys}))
            summary=sess.run(merged,feed_dict={x: batch_train_xs,y:batch_train_ys})
            writer_train.add_summary(summary,step)
            
            
            dataSet = dataTest.next_batch(batch_size)
            batch_test_xs = np.array(dataSet.drop(['label'],axis = 1))
            batch_test_xs = batch_test_xs.reshape([-1,n_steps,n_inputs])
            batch_test_ys = np.array(dataSet['label'])
            
            print('Test accuracy: ',sess.run(accuracy,feed_dict = {x: batch_test_xs,y: batch_test_ys}))
            summary=sess.run(merged,feed_dict = {x:batch_test_xs,y:batch_test_ys})
            writer_test.add_summary(summary,step)
        step+=1    

    saver.save(sess,pathSaveModel)

D:\software\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step0:
Train accuracy:  0.1966
Test accuracy:  0.1982
step10:
Train accuracy:  0.6788
Test accuracy:  0.682
step20:
Train accuracy:  0.8354
Test accuracy:  0.8316
step30:
Train accuracy:  0.8426
Test accuracy:  0.8476
step40:
Train accuracy:  0.8436
Test accuracy:  0.841
step50:
Train accuracy:  0.8744
Test accuracy:  0.8766
step60:
Train accuracy:  0.8664
Test accuracy:  0.862
step70:
Train accuracy:  0.8892
Test accuracy:  0.8954
step80:
Train accuracy:  0.8822
Test accuracy:  0.8814
step90:
Train accuracy:  0.9044
Test accuracy:  0.903
step100:
Train accuracy:  0.898


step1330:
Train accuracy:  0.9204
Test accuracy:  0.9196
step1340:
Train accuracy:  0.9204
Test accuracy:  0.9182
step1350:
Train accuracy:  0.9206
Test accuracy:  0.9212
step1360:
Train accuracy:  0.9188
Test accuracy:  0.9194
step1370:
Train accuracy:  0.9238
Test accuracy:  0.9216
step1380:
Train accuracy:  0.9086
Test accuracy:  0.9146
step1390:
Train accuracy:  0.9194
Test accuracy:  0.9226
step1400:
Train accuracy:  0.924
Test accuracy:  0.9178
step1410:
Train accuracy:  0.9178
Test accuracy:  0.9258
step1420:
Train accuracy:  0.915
Test accuracy:  0.9158
step1430:
Train accuracy:  0.9226
Test accuracy:  0.9214
step1440:
Train accuracy:  0.925
Test accuracy:  0.9256
step1450:
Train accuracy:  0.9206
Test accuracy:  0.9132
step1460:
Train accuracy:  0.9128
Test accuracy:  0.9158
step1470:
Train accuracy:  0.9262
Test accuracy:  0.923
step1480:
Train accuracy:  0.9242
Test accuracy:  0.9222
step1490:
Train accuracy:  0.9214
Test accuracy:  0.9248
step1500:
Train accuracy:  0.9258
T

step2780:
Train accuracy:  0.9258
Test accuracy:  0.9254
step2790:
Train accuracy:  0.9258
Test accuracy:  0.9266
step2800:
Train accuracy:  0.9266
Test accuracy:  0.9212
step2810:
Train accuracy:  0.925
Test accuracy:  0.926
step2820:
Train accuracy:  0.9224
Test accuracy:  0.9248
step2830:
Train accuracy:  0.9114
Test accuracy:  0.92
step2840:
Train accuracy:  0.928
Test accuracy:  0.9242
step2850:
Train accuracy:  0.9286
Test accuracy:  0.9228
step2860:
Train accuracy:  0.9268
Test accuracy:  0.926
step2870:
Train accuracy:  0.9286
Test accuracy:  0.9236
step2880:
Train accuracy:  0.9252
Test accuracy:  0.928
step2890:
Train accuracy:  0.9248
Test accuracy:  0.9254
step2900:
Train accuracy:  0.9178
Test accuracy:  0.9166
step2910:
Train accuracy:  0.9162
Test accuracy:  0.9212
step2920:
Train accuracy:  0.9208
Test accuracy:  0.9254
step2930:
Train accuracy:  0.9274
Test accuracy:  0.9242
step2940:
Train accuracy:  0.9272
Test accuracy:  0.922
step2950:
Train accuracy:  0.9286
Test 